<a href="https://colab.research.google.com/github/ByungjunKim/DDMKL/blob/main/240411_RissScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RISS에서 학위 논문 데이터 수집하기

In [ ]:
import requests
import math
import pickle
import time
import sys
import pandas as pd
from tqdm.notebook import tqdm
from random import uniform
import lxml
import lxml.etree as et
from bs4 import BeautifulSoup
import glob
import urllib3
urllib3.disable_warnings()
from natsort import natsorted
import re
import concurrent.futures
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
    "DNT": "1", # Do Not Track 요청 헤더 (사용자 추적 거부)
    # 필요한 추가 헤더를 여기에 추가
}

In [ ]:
def request_until_success(url, headers, timeout=7, delay=3):
    """
    Continuously makes a request to the specified URL with a timeout until the request is successful.

    Args:
    - url (str): The URL to request.
    - timeout (int): The timeout for the request in seconds.
    - delay (int): The delay between retries in seconds.

    Returns:
    - response: The successful response from the server.
    """
    attempt = 1
    while True:
        try:
            # print(f"Attempt {attempt}")
            response = requests.get(url,headers=headers, timeout=timeout)
            response.raise_for_status()  # Raises HTTPError for bad responses
            # print("Request successful")
            return response
        except requests.Timeout as e:
            print(f"Request timed out: {e}")
        except requests.RequestException as e:
            print(f"Request failed: {e}")

        # print(f"Retrying in {delay} seconds...")
        time.sleep(delay)
        attempt += 1

### 검색 조건에 맞는 URL 설정
학과정보 : 국어국문 OR 국문  
학위수여연도 : 2000~2023  
학위유형 : 국내박사  
페이지당 출력 : 100개씩  
https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&iStartCount=0&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query=

In [ ]:
# 검색 조건을 넣은 url을 아래에 복사해 넣으세요
url = 'https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&strQuery=&exQuery=&exQueryText=&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&iStartCount=0&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query='

In [ ]:
req = request_until_success(url,headers).text

In [ ]:
soup = BeautifulSoup(req, 'lxml')

In [ ]:
cont_ml60_classes = soup.find_all('div', class_='cont ml60')

data = [{
    'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
    'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
    'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
    'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
    'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
    'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
    'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
} for cont_ml60 in cont_ml60_classes]

In [ ]:
data

[{'title': '외국인 유학생 대상국어국문학 전공 기초 교육과정 개발 연구',
  'link': '/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=3999e8d77c9617e7ffe0bdc3ef48d419&keyword=',
  'writer': '임형옥',
  'assigned': '고려대학교 대학원',
  'year': '2020',
  'grad': '국내박사',
  'preAbstract': 'The aim of this study is to develop a preparation curriculum for Korean language and literature major international students. International students who study in Korean universities often have difficulties when they take core subjects, mainly due to lack of essential knowledge in their major or incompetent task performance. If it is the case, a major preparation curriculum may reduce such obstacles and help the students feel confident in their major subjects. In fact, majoring in the Korean language and literature needs far higher command of Korean than others, which implies that international students majoring in the Korean language and literature require a high level of Korean proficiency and competence in specific lan

### 기본 정보 자동으로 수집하기

In [ ]:
# Assuming 'soup' is your BeautifulSoup object
num_span = soup.find('span', class_='num')

if num_span is not None:
    total_count = int(num_span.get_text().replace(',', ''))
else:
    total_count = 0

total_count

4812

In [ ]:
# The base URL without the 'iStartCount' parameter
# Parse the URL
parsed_url = urlparse(url)

# Parse the query parameters
params = parse_qs(parsed_url.query)

# Remove the 'iStartCount' parameter
params.pop('iStartCount', None)

# Re-encode the query parameters
new_query = urlencode(params, doseq=True)

# Replace the old query parameters with the new ones
new_url = parsed_url._replace(query=new_query)

# Unparse the URL
base_url = urlunparse(new_url)

print(base_url)

https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor%2C%EA%B5%AD%EC%96%B4%EA%B5%AD%EB%AC%B8%40op%2COR%40znMajor%2C%EA%B5%AD%EB%AC%B8&order=%2FDESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&mat_subtype=T2&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y


##### 실제 수집 코드

In [ ]:
# The base URL without the 'iStartCount' parameter
# base_url = "https://www.riss.kr/search/Search.do?isDetailSearch=Y&searchGubun=true&viewYn=OP&queryText=znMajor,국어국문@op,OR@znMajor,국문&strQuery=&exQuery=&exQueryText=&order=/DESC&onHanja=false&strSort=RANK&p_year1=2000&p_year2=2023&orderBy=&mat_type=&mat_subtype=T2&fulltext_kind=&t_gubun=&learning_type=&ccl_code=&inside_outside=&fric_yn=&db_type=&image_yn=&gubun=&kdc=&ttsUseYn=&l_sub_code=&fsearchMethod=search&sflag=1&isFDetailSearch=N&pageNumber=1&resultKeyword=&fsearchSort=&fsearchOrder=&limiterList=&limiterListText=&facetList=&facetListText=&fsearchDB=&icate=bib_t&colName=bib_t&pageScale=100&isTab=Y&regnm=&dorg_storage=&language=&language_code=&clickKeyword=&relationKeyword=&query="

# Calculate the number of iterations
iterations = math.ceil(total_count / 100)

# Initialize an empty list to store the data
data = []

def fetch_data(i):
    # Create the URL with the incremented 'iStartCount' parameter
    url = f"{base_url}&iStartCount={i * 100}"

    # Send a GET request to the URL
    response = request_until_success(url, headers)

    # Create a BeautifulSoup object from the response text
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all 'div' elements with class 'cont ml60'
    cont_ml60_classes = soup.find_all('div', class_='cont ml60')

    # Extract the data and return it
    return [{
        'title': cont_ml60.find('p', class_='title').get_text(strip=True) if cont_ml60.find('p', class_='title') else None,
        'link': cont_ml60.find('p', class_='title').find('a').get('href') if cont_ml60.find('p', class_='title') and cont_ml60.find('p', class_='title').find('a') else None,
        'writer': cont_ml60.find('p', class_='etc').find_all('span')[0].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 0 else None,
        'assigned': cont_ml60.find('p', class_='etc').find_all('span')[1].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 1 else None,
        'year': cont_ml60.find('p', class_='etc').find_all('span')[2].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 2 else None,
        'grad': cont_ml60.find('p', class_='etc').find_all('span')[3].get_text(strip=True) if cont_ml60.find('p', class_='etc') and len(cont_ml60.find('p', class_='etc').find_all('span')) > 3 else None,
        'preAbstract': cont_ml60.find('p', class_='preAbstract').get_text(strip=True) if cont_ml60.find('p', class_='preAbstract') else None
    } for cont_ml60 in cont_ml60_classes]

# Loop over the range, incrementing by 100 each time
with concurrent.futures.ThreadPoolExecutor() as executor:
    data_list = list(tqdm(executor.map(fetch_data, range(iterations)), total=iterations))

# Flatten the list of lists
data = [item for sublist in data_list for item in sublist]

  0%|          | 0/49 [00:00<?, ?it/s]

Request failed: ('Connection broken: IncompleteRead(1338 bytes read, 6718 more expected)', IncompleteRead(1338 bytes read, 6718 more expected))
Request failed: ('Connection broken: IncompleteRead(2563 bytes read, 5494 more expected)', IncompleteRead(2563 bytes read, 5494 more expected))
Request failed: ('Connection broken: IncompleteRead(5205 bytes read, 2852 more expected)', IncompleteRead(5205 bytes read, 2852 more expected))
Request failed: ('Connection broken: IncompleteRead(2312 bytes read, 5745 more expected)', IncompleteRead(2312 bytes read, 5745 more expected))


In [ ]:
df = pd.DataFrame(data)
df

,title,link,writer,assigned,year,grad,preAbstract
0,외국인 유학생 대상국어국문학 전공 기초 교육과정 개발 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,임형옥,고려대학교 대학원,2020,국내박사,The aim of this study is to develop a preparat...
1,중국 내 한국어학과 문학 교육과정의 설계와 재구성 연구 : 길림성 소재 대학의 고전...,/search/detail/DetailView.do?p_mat_type=be54d9...,정반반,공주대학교 일반대학원,2021,국내박사,이 글은 중국 길림성 소재 대학의 한국어학과 문학교육과정에 관한 내용이다. 1972...
2,인문계열 학과 법제 교육 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,김태림,경희대학교 대학원,2023,국내박사,The purpose of this study is to emphasize the ...
3,"한국문학 영어번역의 번역비평적 비교연구 : 신경숙의 <엄마를 부탁해>, 한강의 <채...",/search/detail/DetailView.do?p_mat_type=be54d9...,심선향,韓國外國語大學校 大學院,2021,국내박사,본 연구는 세계로 발돋움하고 있는 한국문학의 중심에 선 세 편의 작품 신경숙의 『엄...
4,러시아에서의 한국문학 수용 과정 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,한승희,한양대학교 대학원,2021,국내박사,None
...,...,...,...,...,...,...,...
4807,베이징 공간 재현의 변천 양상 연구 : '京味'와 '京味文學'에 대한 고찰을 중심으로,/search/detail/DetailView.do?p_mat_type=be54d9...,이수연,慶熙大學校,2019,국내박사,유구한 역사를 가진 베이징은 원(元)왕조 이후 현재까지 수도의 지위를 유지하고 있는...
4808,陳映眞 小說 硏究,/search/detail/DetailView.do?p_mat_type=be54d9...,주재희,韓國外國語大學校,2002,국내박사,None
4809,初期 魯迅의 近代的 思想 硏究,/search/detail/DetailView.do?p_mat_type=be54d9...,신홍철,韓國外國語大學校,2002,국내박사,None
4810,중국의 탈놀이 <<大頭和尙>> 연구,/search/detail/DetailView.do?p_mat_type=be54d9...,김순희,成均館大學校,2005,국내박사,None


### 논문별 상세정보 수집하기

In [ ]:
df['link'] = 'https://www.riss.kr' + df['link']
df['link']

0       https://www.riss.kr/search/detail/DetailView.d...
1       https://www.riss.kr/search/detail/DetailView.d...
2       https://www.riss.kr/search/detail/DetailView.d...
3       https://www.riss.kr/search/detail/DetailView.d...
4       https://www.riss.kr/search/detail/DetailView.d...
                              ...                        
4807    https://www.riss.kr/search/detail/DetailView.d...
4808    https://www.riss.kr/search/detail/DetailView.d...
4809    https://www.riss.kr/search/detail/DetailView.d...
4810    https://www.riss.kr/search/detail/DetailView.d...
4811    https://www.riss.kr/search/detail/DetailView.d...
Name: link, Length: 4812, dtype: object

In [ ]:
headers_for_riss = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7,ja;q=0.6",
    "Connection": "keep-alive",
    "Cookie": "Your Cookie Here",
    "Host": "www.riss.kr",
    "Referer": None,
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "sec-ch-ua": '"Google Chrome";v="123", "Not:A-Brand";v="8", "Chromium";v="123"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "Windows"
}

In [ ]:
def fetch_data(link):
    headers_for_riss["Referer"] = link
    # response = requests.get(link, headers=headers)
    response = request_until_success(link, headers_for_riss)
    soup = BeautifulSoup(response.text, 'lxml')
    data = {}
    data['link'] = link
    for li in soup.find('div',class_='infoDetailL').find_all('li'):
        key_element = li.find('span', {'class': 'strong'})
        value_element = li.find('div')
        if key_element and value_element:
            key = key_element.text.strip()
            value = value_element.text.strip()
            data[key] = value
    try:
        text_off_divs = soup.find('div', class_='content additionalInfo').find_all('div', class_='text off')
        title_text_dict = {}
        for div in text_off_divs:
            title = div.find_previous_sibling('p', class_='title')
            if title:
                if title.text.strip() == "참고문헌 (Reference)":
                    title_text_dict[title.text.strip()] = [p.text.strip() for p in div.find_all('p')]
                else:
                    title_text_dict[title.text.strip()] = div.text.strip()
        data.update(title_text_dict)
    except:
        print(f'참고문헌 등 추가 정보 없음 :{link}')
    return data

In [ ]:
# 실행 코드
links = df['link'].tolist()[:100] # 100개만 테스트 다하려면 [:100] 제거
with concurrent.futures.ThreadPoolExecutor() as executor:
    data_list = list(tqdm(executor.map(fetch_data, links), total=len(links)))

  0%|          | 0/100 [00:00<?, ?it/s]

참고문헌 등 추가 정보 없음 :https://www.riss.kr/search/detail/DetailView.do?p_mat_type=be54d9b8bc7cdb09&control_no=c458192d34e99581&keyword=


In [ ]:
detail = pd.DataFrame(data_list)
detail

,link,저자,발행사항,학위논문사항,발행연도,작성언어,주제어,발행국(도시),기타서명,형태사항,일반주기명,UCI식별코드,DOI식별코드,소장기관,다국어 초록 (Multilingual Abstract),목차 (Table of Contents),참고문헌 (Reference),DDC,국문 초록 (Abstract),KDC
0,https://www.riss.kr/search/detail/DetailView.d...,임형옥,"서울 : 고려대학교 대학원, 2020",학위논문(박사)-- 고려대학교 대학원: 국어국문학과 2020. 8,2020,한국어,특정 학문 목적 한국어교육\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t...,서울,A study on the development of a preparation cu...,"vii, 239 p. ; 26 cm",부록수록지도교수: 김정숙참고문헌: p. 205-214,I804:11009-000000231531,10.23186/korea.000000231531.11009.0001168,고려대학교 도서관,The aim of this study is to develop a preparat...,목 차\n\n1. 서론1\n1.1. 연구의 목적 및 필요성1\n1.2. 선행연구 검...,"[1 이익섭, ""?국어학개설?"", 학연사, 2011, 2 신승용, ""「국어학 개...",NaN,NaN,NaN
1,https://www.riss.kr/search/detail/DetailView.d...,정반반\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\...,"공주 : 공주대학교 일반대학원, 2021",학위논문(박사)-- 공주대학교 일반대학원 : 국어국문학과 2022. 2,2021,한국어,"중국 한국어학과, 한국문학교육과정, 고전문학",충청남도,A Study on the Design and Reconstruction of Ko...,183p. 26cm,지도교수:권대광참고문헌 : p.164-170,I804:44004-000000031401,NaN,국립공주대학교 도서관,NaN,I. 서론 1\n 1. 연구의 필요성 및 목적 1\n 2. 선행연구 검토 5\n 3...,NaN,811 판사항(22),이 글은 중국 길림성 소재 대학의 한국어학과 문학교육과정에 관한 내용이다. 1972...,NaN
2,https://www.riss.kr/search/detail/DetailView.d...,김태림,"서울 : 경희대학교 대학원, 2023",학위논문(박사)-- 경희대학교 대학원 : 국어국문학과 2023. 2,2023,한국어,"인문학, 국어국문학, 법제 교육, 저작권, 지식재산권",서울,A Study on Legal Education of Humanities Depar...,"vii, 122 p. : 삽화 ; 26 cm",경희대학교 논문은 저작권에 의해 보호받습니다.지도교수: 이정재참고문헌: p. 108...,I804:11006-200000659401,NaN,경희대학교 중앙도서관,The purpose of this study is to emphasize the ...,<국문초록>ⅰ\n\nⅠ. 서론 1\n제1절 연구의 필요성 및 목적 1\n제2절 연구...,NaN,811 판사항(22),「인문계열 학과 법제 교육 연구」는 인문학 학습자들이 사회 현장에서 겪는 실제적인 ...,NaN
3,https://www.riss.kr/search/detail/DetailView.d...,심선향,"서울 : 韓國外國語大學校 大學院, 2021",학위논문(박사) -- 韓國外國語大學校 大學院 국어국문학과 2021. 2,2021,한국어,문학번역\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\r\n\t...,서울,A comparative study on the translation critici...,"vi, 279 p. : 삽도 ; 26 cm",한국외국어대학교 논문은 저작권에 의해 보호받습니다.지도교수: 채호석참고문헌: p. ...,I804:11059-200000365414,NaN,한국외국어대학교 글로벌캠퍼스 도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t...,This study was intended to present productive ...,Ⅰ. 서론 1\n 1. 연구 배경 및 목적 1\n 1.1. 연구 배경·의의 ...,NaN,895.1 판사항(22),본 연구는 세계로 발돋움하고 있는 한국문학의 중심에 선 세 편의 작품 신경숙의 『엄...,NaN
4,https://www.riss.kr/search/detail/DetailView.d...,한승희,"서울 : 한양대학교 대학원, 2021",학위논문(박사)-- 한양대학교 대학원 : 국어국문학과 2021. 2,2021,한국어,한국문학,서울,NaN,"ii, 174 p. : 삽도 ; 26 cm.","권두 국문요지, 권말 Abstract 수록지도교수: 유성호참고문헌: p. 156-172",I804:11062-200000485955,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,NaN,[ 목차 ]\n\n\n국문요지\t Ⅱ\n\n제1장 서론\t 1\n제1절 연구목적과 ...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,https://www.riss.kr/search/detail/DetailView.d...,김수정,"경산 : 영남대학교 대학원, 2019",학위논문(박사)-- 영남대학교 대학원 : 국어국문학과 국어학전공 2019.2,2019,한국어,NaN,경상북도,NaN,167 p. : 도표 ; 26 cm,지도교수: 최동주영남대학교 논문은 저작권에 의해 보호받습니다,I804:47017-200000182796,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,The purposes of this thesis are to classify Ko...,1. 서론 1\n 1.1. 연구 목적과 논의의 구성 1\n 1.2. 선행 연구 3\...,"[1 최현배, ""우리말본"", 정음사, 1937, 2 허 웅, ""「우리옛말본」"",...",NaN,"본 연구는 접속 영역의 관점에서 국어 접속문의 유형을 분류하고, 접속문이 어느 유형...",050 판사항(6)
96,https://www.riss.kr/search/detail/DetailView.d...,김기석,"서울 : 고려대학교 대학원, 2002",학위논문(박사)-- 고려대학교 대학원: 국어국문학과 2003. 2,2002,한국어,멀티미디어\r\n\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\r\n\...,서울,NaN,"x, 337p. : 삽도 ; 26cm.",참고문헌: p. 263-298,NaN,NaN,고려대학교 도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t...,NaN,목차 = v\r\n국문초록 = i\r\n1. 서론 = 11\r\n 1.1 연구의 목...,NaN,NaN,이 연구는 국민공통 7-10학년 과정에서 멀티미디어를 활용한 국어 사용기능의 교수 ...,307 판사항(4)
97,https://www.riss.kr/search/detail/DetailView.d...,오혜정,"서울 : 한양대학교 대학원, 2015",학위논문(박사)-- 한양대학교 대학원 : 국어국문학과 2015. 8,2015,한국어,한국문학,서울,NaN,"ii, 167 p. ; 26 cm.","지도교수: 이재복권두 국문요지, 권말 Abstract 수록참고문헌: p. 157-162",NaN,NaN,국립중앙도서관 \r\n\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t...,NaN,□ 국문요지\r\n \nⅠ. 서론\t1\r\n1. 문제 제기 및 연구사 검토\t1\...,NaN,NaN,NaN,NaN
98,https://www.riss.kr/search/detail/DetailView.d...,변이주,"군산 : 군산대학교, 2014",학위논문(박사)-- 군산대학교 : 국어국문학

In [ ]:
# csv로 저장
detail.to_csv('riss_detail.csv', index=False)